<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# _*Creating a third party provider*_ 

The latest version of this notebook is available on https://github.com/qiskit/qiskit-tutorial.

***
#### Contributors


#### Affiliations 
*[1] IBM Q*

Qiskit provides several in-house simulators, which simulate quantum circuits on a classical computer. These simulators differ in their implementation languages and wave function representations. All the in-house simulators work on circuits that are written in the QASM language, or any other API supported by Qiskit. Moreover, they all inherit from the same base class ``BaseBackend``. This allows polymorphism between simulators. For example, one can write a function which runs a simulator and analyzes its results, then call this function for different simulators.

The term "backend" refers to any simulator or device, capable of reading a quantum circuit, executing it (on either a classical or quantum device), and displaying the execution results. Moreover, we require that the quantum circuit and execution results adhere to the format of Qiskit API.

The in-house simulators of Qiskit are naturally backends. External simulators, on the other hand, usually have their own formats for quantum circuits and simulation results. This tutorial explains how to create a Qiskit backend, which wraps an external simulator. This allows to run the external simulator, while enjoying the benefits of polymorphism and uniform API across simulators.

# An external simulator for this tutorial

We shall construct a very simple simulator. The simulator accepts only a single quantum circuit, where all the gates are Hadamard gates, and all qubits are measured at the end. The input format is a list of qubits on whom Hadamard gates are applied. The simulator returns the counts of each basis state, in the form of a list, where the basis states are assumed to be ordered lexicographically.

In [1]:
def run_hadamard_simulator(number_of_qubits, list_of_qubits, shots):
    '''
    Run our amazing Hadamard simulator!
    Note: this function is not designed to be efficient
    
    Args:
        number_of_qubits (integer): number of qubits in the qunatum circuit
        list_of_qubits (list of integers): a list of qubits on whom Hadamard gates are applied
        shots (integer): number of shots

    Returns:
        list of integers:
            each entry in the list contains the number of shots 
            where the measurement result is the correspnding basis state;
            basis states are ordered lexicographically
    '''
    
    # For each qubit, store whether it is manipulated by an odd number of Hadamard gates
    # Example: for run_hadamard_simulator(5, [3, 1, 3, 4], 100)
    # we obtain hadamard_list:
    # [0, 1, 0, 0, 1]
    # because qubits 1 and 4 have an odd number of Hadamard gates.
    hadamard_list = [0]*number_of_qubits
    for qubit in list_of_qubits:
        hadamard_list[qubit] = (1 + hadamard_list[qubit])%2
    
    # Calculate the result for each basis state
    result = [0]*(2**number_of_qubits)
    for i in range(2**number_of_qubits):
        # Example: when i is 2, 
        # the basis_state is 00010
        basis_state = '{0:b}'.format(i).zfill(number_of_qubits)
        
        for qubit in range(number_of_qubits):
            if hadamard_list[qubit] == 0 and basis_state[qubit] == '1':
                result[i] = 0
                break
            if hadamard_list[qubit] == 1:
                result[i] += shots/(2**(1 + hadamard_list.count(1)))
                
    return result


run_hadamard_simulator(4, [3, 1, 3, 2], 1024)

[256.0, 0, 256.0, 0, 256.0, 0, 256.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Creating a job class

TODO: complete after decision what to include in this section.

In [2]:
from qiskit.backends import BaseJob

class HadamardJob(BaseJob):
    def __init__(self, backend):
        super().__init__(backend, 1)
        
    def result(self):
        return self._result
    
    def cancel(self):
        pass
    
    def status(self):
        pass
    
    def submit(self):
        pass

# Creating a backend

Steps of backend creation:
- Define a new class, which we call ``HadamardSimulator``.
  The new class must inherit from ``BaseBackend`` 
  (``import qiskit.backends`` for ``BaseBackend`` to be recognized). 
  Follow the instructions in ``qiskit/backends/basebackend.py``.
- The function ``run`` receives a ``QObj`` object and returns a ``Result`` object. Internal steps therein:  
  - Translate the circuit, given in the ``QObj`` object, to the Hadamard simulator's language.
  - Run the Hadamard simulator. 
  - Create a ``Result`` object, and populate it with the Hadamard simulator's results.
    Follow the instructions given in the constructor of ``Result``, in ``qiskit/result/_result.py``.
    
 This is what it looks like:

In [3]:
from qiskit.backends import BaseBackend
from qiskit import qobj as qiskit_qobj
from qiskit import Result


class HadamardSimulator(BaseBackend):
    '''
    A wrapper backend for the Hadamard simulator
    '''

    def __init__(self, provider=None):
        configuration = {
            'name': 'hadamard_simulator',
            'simulator': True,
            'local': True,
            'description': 'Simulates only Hadamard gates',
            'basis_gates': ['h', 'x'],  # basis_gates must contain at least two gates
            'coupling_map': 'all-to-all'
        }
        
        # We will explain about the provider in the next section
        super().__init__(configuration, provider=provider)


    def run(self, qobj):
        """Run qobj

        Args:
            qobj (QObj): circuit description

        Returns:
            HadamardJob: derived from BaseJob
        """
        hadamard_job = HadamardJob(None)
            
        experiment_results = []
        for circuit_index, circuit in enumerate(qobj.experiments):
            number_of_qubits = circuit.header.number_of_qubits
            shots = qobj.config.shots
            
            list_of_qubits = []
            for operation in circuit.instructions:
                if getattr(operation, 'conditional', None):
                    raise QiskitError('conditional operations are not supported '
                                      'by the Hadamard simulator')
                if operation.name != 'h':
                    if operation.name == 'measure':
                        continue
                    else:
                        raise QiskitError('The Hadamrd simulator allows only Hadamard gates')
                
                list_of_qubits.append(operation.qubits[0])
            
            # Need to verify that 
            # all the qubits are measured, and to different classical registers.
            # Raise an error otherwise.
            # We skip this part here.            
            
            counts = run_hadamard_simulator(number_of_qubits, list_of_qubits, shots)
            
            formatted_counts = {}
            for i in range(2**number_of_qubits):
                basis_state = '{0:b}'.format(i).zfill(number_of_qubits)
                formatted_counts[basis_state] = counts[i]
                           
            experiment_results.append(qiskit_qobj.ExperimentResult(
                success=True, 
                shots=[1, shots], 
                data={'counts': formatted_counts},
                header={'name': circuit.header.name}))
                        
        hadamard_job._result = Result(qiskit_qobj.Result(
            results = experiment_results,
            backend_name = 'hadamard_simulator',
            backend_version = '0',
            qobj_id = 0,
            job_id = 1,
            success = True))
        
        return hadamard_job

Congratulations, your backend is ready! Now you can create a Qiskit circuit and run your simulator:

In [4]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute

qreg = QuantumRegister(4)
creg = ClassicalRegister(4)
qc = QuantumCircuit(qreg, creg)
qc.h(qreg[3])
qc.h(qreg[1])
qc.h(qreg[3])
qc.h(qreg[2])
qc.measure(qreg, creg)

hadamard_job = execute(qc, backend=HadamardSimulator(), skip_transpiler=True, shots=1024)
result = hadamard_job.result()

print(result.get_counts(qc))

{'0000': 256.0, '0001': 0, '0010': 256.0, '0011': 0, '0100': 256.0, '0101': 0, '0110': 256.0, '0111': 0, '1000': 0, '1001': 0, '1010': 0, '1011': 0, '1100': 0, '1101': 0, '1110': 0, '1111': 0}


# Creating a provider

The provider allows you to group several simulators together. Then you can execute one of the simulators by entering the backend's name. To achieve the capability, create a class which inherits from ``BaseProvider``. The new class must implement the methods ``get_backend`` and ``available_backends``.

In [5]:
from qiskit.backends import BaseProvider
from qiskit.backends.providerutils import filter_backends

class HadamardProvider(BaseProvider):
    """Provider for the Hadamard backend"""

    def __init__(self, *args, **kwargs):
        super().__init__(args, kwargs)

        # Populate the list of Hadamard backends
        self._backends = [HadamardSimulator(provider=self)]

    def get_backend(self, name=None, **kwargs):
        return super().get_backend(name=name, **kwargs)

    def backends(self, name=None, filters=None, **kwargs):
        # pylint: disable=arguments-differ
        if name:
            kwargs.update({'name': name})

        return filter_backends(self._backends, filters=filters, **kwargs)

    def __str__(self):
        return 'HadamardProvider'

The following piece of code runs two simulators on the same quantum circuit. The simulators are accessed by their providers.

In [6]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit.backends.aer import AerProvider

hadamard_provider = HadamardProvider()
aer_provider = AerProvider()

new_hadamard_job = execute(qc, hadamard_provider.get_backend('hadamard_simulator'), skip_transpiler=True, shots=1024)
new_hadamard_result = new_hadamard_job.result()

aer_job = execute(qc, aer_provider.get_backend('qasm_simulator'), skip_transpiler=True, shots=1024)
aer_result = new_hadamard_job.result()

print("Hadamard simulator: ", new_hadamard_result)
print(new_hadamard_result.get_counts(qc))
print("Aer simulator: ", aer_result)
print(aer_result.get_counts(qc))

Hadamard simulator:  SUCCESS = True
{'0000': 256.0, '0001': 0, '0010': 256.0, '0011': 0, '0100': 256.0, '0101': 0, '0110': 256.0, '0111': 0, '1000': 0, '1001': 0, '1010': 0, '1011': 0, '1100': 0, '1101': 0, '1110': 0, '1111': 0}
Aer simulator:  SUCCESS = True
{'0000': 256.0, '0001': 0, '0010': 256.0, '0011': 0, '0100': 256.0, '0101': 0, '0110': 256.0, '0111': 0, '1000': 0, '1001': 0, '1010': 0, '1011': 0, '1100': 0, '1101': 0, '1110': 0, '1111': 0}
